Networks - Final Project

"Is there an optimal road or road structure to close and convert to a public use space in Barcelona that minimizes the impact on traffic?"

-Clarice Mottet, Amber Walker, Mox Ballo

General Notes:

There are only 930 nodes in the link_file, there are 2522 edges in the link_file (haven't checked for duplicates). There are no duplicate edges (based on origin_node to to_node)

website link that the frank wolfe code is from

https://nbviewer.org/github/PyTrans/Urban-Network-Analysis/blob/master/Trip_Assignment-Frank-Wolfe_Algorithm.ipynb

website link that the TransportationNetworks code is from

https://github.com/PyTrans/Urban-Network-Analysis/tree/master/pytrans/UrbanNetworkAnalysis

SUPER IMPORTANT

I saw in the TransportationNetworks.py file that their initial alpha was .5 but everywhere said it should be .15, so I changed it to .15 in my file.

Because there is no node_file and I have to create it, I had to alter some of the TransportationNetworks.py code

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
#libraries

import pandas as pd
import numpy as np
import networkx as nx
import networkx as nx
import scipy.integrate as integrate 
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
from scipy.misc import derivative

# import os
# os.chdir('/content/drive/My Drive/networks_final_project_collab/networks_finalproject')

import TransportationNetworks_team as tn

# path_in_ = r'./inputs/'
# path_out_ = r'./outputs'
# path_out_results_ = r'./outputs/results'

path_in_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/inputs/'
path_out_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/outputs'
path_out_results_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/outputs/results'

In [3]:
#functions

#from website to calculate latency
def BPR(t0, xa, ca, alpha, beta):
    ta = t0*(1+alpha*(xa/ca)**beta)
    return ta

#from website: method to calculate nash equilibrium (when SO = False)
def calculateZ(theta, network, SO):
    z = 0
    for linkKey, linkVal in network.items():
        t0 = linkVal['t0']
        ca = linkVal['capa']
        beta = linkVal['beta']
        alpha = linkVal['alpha']
        aux = linkVal['auxiliary'][-1]
        flow = linkVal['flow'][-1]
        
        if SO == False:
            z += integrate.quad(lambda x: BPR(t0, x, ca, alpha, beta), 0, flow+theta*(aux-flow))[0]
        elif SO == True:
            z += list(map(lambda x : x * BPR(t0, x, ca, alpha, beta), [flow+theta*(aux-flow)]))[0]
    return z

#from website: finds nash equilibrium
def lineSearch(network, SO):
    theta = minimize_scalar(lambda x: calculateZ(x, network, SO), bounds = (0,1), method = 'Bounded')
    return theta.x

In [4]:
#functions - user, based on jupyter notebook

def initialize(link_file, trip_file, od_vols, origins, od_dic, links):
    #set objective and open the network
    SO = False # True - System optimum, False - User equilibrium
    networkSubset = tn.Network(link_file, trip_file, od_vols, origins, od_dic, links)

    #team - start of initialization

    # define output variables, network and fwResult
    network = {(u,v): {'t0':d['object'].t0, 'alpha':d['object'].alpha, \
            'beta':d['object'].beta, 'capa':d['object'].capacity, 'flow':[], \
            'auxiliary':[], 'cost':[]} for (u, v, d) in networkSubset.graph.edges(data=True)}

    fwResult = {'theta':[], 'z':[]}

    # initial all-or-nothing assignment and update link travel time(cost)
    networkSubset.all_or_nothing_assignment()
    networkSubset.update_linkcost()

    for linkKey, linkVal in network.items():
        linkVal['cost'].append(networkSubset.graph[linkKey[0]][linkKey[1]]['weight'])
        linkVal['auxiliary'].append(networkSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)
        linkVal['flow'].append(networkSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)

    return networkSubset, network, fwResult

#calculate nash equilibrium
def calc_nash(networkSubset, network, fwResult):
    SO=False
    ## iterations
    iterNum=0
    iteration = True
    while iteration:
        iterNum += 1
        networkSubset.all_or_nothing_assignment()
        networkSubset.update_linkcost()
        
        # set auxiliary flow using updated link flow
        for linkKey, linkVal in network.items():
            linkVal['auxiliary'].append(networkSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)
            
        # getting optimal move size (theta)
        theta = lineSearch(network, SO)
        fwResult['theta'].append(theta)
        
        # set link flow (move) based on the theta, auxiliary flow, and link flow of previous iteration
        for linkKey, linkVal in network.items():
            aux = linkVal['auxiliary'][-1]
            flow = linkVal['flow'][-1]
            linkVal['flow'].append(flow + theta*(aux-flow))
            
            networkSubset.graph[linkKey[0]][linkKey[1]]['object'].vol =  flow + theta * (aux - flow)
            networkSubset.graph[linkKey[0]][linkKey[1]]['object'].flow = flow + theta * (aux - flow)
            
        # update link travel time
        networkSubset.update_linkcost()
        
        # calculate objective function value
        z=0
        for linkKey, linkVal in network.items():
            linkVal['cost'].append(networkSubset.graph[linkKey[0]][linkKey[1]]['weight'])
            totalcost = networkSubset.graph[linkKey[0]][linkKey[1]]['object'].get_objective_function()
            z+=totalcost
            
        fwResult['z'].append(z)        
            
        # convergence test
        if iterNum == 1:
            iteration = True
        else:
            if abs(fwResult['z'][-2] - fwResult['z'][-1]) <= 0.001 or iterNum==3000:
                iteration = False

    return networkSubset, network, fwResult

In [5]:
#user - based on TransportationNetworks

def open_trip_file_team(trip_file):
    demand_factor=1.0
    f = open(trip_file)
    lines = f.readlines()
    f.close()

    od_vols = {}
    current_origin = None

    for line in lines:
        if current_origin == None and line.startswith("Origin"):
            origin = str(int(line.split("Origin")[1]))
            current_origin = origin

        elif current_origin != None and len(line) < 3:
            # print "blank",line,
            current_origin = None

        elif current_origin != None:
            to_process = line[0:-2]
            for el in to_process.split(";"):
                try:
                    dest = str(int(el.split(":")[0]))
                    demand = float(el.split(":")[1]) * demand_factor
                    od_vols[current_origin, dest] = demand
                except:
                    continue
    origins = [str(i) for i, j in od_vols]
    origins = list(dict.fromkeys(origins).keys())

    od_dic = {}
    for (origin, destination) in od_vols:
        if origin not in od_dic:
            od_dic[origin] = {}

        od_dic[origin][destination] = od_vols[origin, destination]
    return od_vols, origins, od_dic


def open_link_file_team(link_file):
    SO = False
    link_fields = {"from": 1, "to": 2, "capacity": 3, "length": 4, "t0": 5, \
                    "B": 6, "beta": 7, "V": 8}
    f = open(link_file)
    lines = f.readlines()
    f.close()

    links_info = []

    header_found = False
    for line in lines:
        if not header_found and line.startswith("~"):
            header_found = True
        elif header_found:
            links_info.append(line)

    nodes = {}
    links = []

    for line in links_info:
        data = line.split("\t")

        try:
            origin_node = str(int(data[link_fields["from"]]))
        except IndexError:
            continue
        to_node = str(int(data[link_fields["to"]]))
        capacity = float(data[link_fields["capacity"]])
        length = float(data[link_fields["length"]])
        alpha = float(data[link_fields["B"]])
        beta = float(data[link_fields["beta"]])

        if origin_node not in nodes:
            n = tn.Node(node_id=origin_node)
            nodes[origin_node] = n

        if to_node not in nodes:
            n = tn.Node(node_id=to_node)
            nodes[to_node] = n

        l = tn.Link(link_id=len(links), length=length, capacity=capacity, alpha=alpha, beta=beta,
                    from_node=origin_node, to_node=to_node, flow=float(0.0), SO=SO)

        links.append(l)
    return links
    
def build_datastructure_team(links):        
    graph = nx.DiGraph()

    for l in links:
        graph.add_edge(l.get_from_node(), l.get_to_node(), object=l, time=l.get_time())
    return graph

In [6]:
#functions - user

#remove edges from consideration

#create mini block list

#create super block list

#remove mini block edges from net file

#remove mini block trips from trips file
#(if the node is within the mini block it has to be removed 
#from the origin and destinactions trips file)

#calc total latency

#store and export results

In [7]:
#import data into programming environment
directory = path_in_
link_file = '{}Barcelona_net.tntp'.format(path_in_)
trip_file = '{}Barcelona_trips.tntp'.format(path_in_)
# node_file = '{}networkSubset_node.tntp'.format(directory)

od_vols, origins, od_dic = open_trip_file_team(trip_file)
links = open_link_file_team(link_file)
graph = build_datastructure_team(links)
#to be able to access stuff in graph

#subset to just one neighborhood of Barcelona

# networkSubset = tn.Network(link_file, trip_file, SO=False)
# networkSubset = tn.Network(link_file, trip_file, od_vols, origins, od_dic, links, graph)

In [8]:
print(graph.edges())

[(1, 290), (1, 307), (1, 316), (290, 1), (290, 276), (290, 289), (307, 1), (307, 308), (307, 311), (307, 312), (316, 1), (316, 300), (316, 311), (316, 314), (2, 302), (2, 304), (302, 2), (302, 301), (302, 303), (302, 304), (304, 2), (304, 76), (304, 79), (304, 80), (304, 85), (304, 90), (304, 302), (304, 303), (304, 1020), (3, 301), (3, 306), (301, 3), (301, 4), (301, 302), (301, 306), (306, 3), (306, 4), (306, 301), (306, 308), (306, 310), (306, 1020), (4, 301), (4, 306), (5, 298), (5, 299), (5, 310), (298, 5), (298, 274), (298, 297), (299, 5), (299, 288), (299, 297), (299, 300), (310, 5), (310, 278), (310, 306), (310, 309), (6, 424), (6, 425), (6, 456), (424, 6), (424, 423), (424, 427), (424, 452), (425, 6), (425, 279), (425, 428), (456, 6), (456, 8), (456, 488), (456, 489), (7, 274), (7, 281), (7, 283), (274, 7), (274, 280), (274, 282), (274, 360), (274, 433), (281, 202), (281, 7), (281, 272), (281, 284), (283, 7), (283, 280), (283, 284), (283, 437), (8, 272), (8, 456), (8, 474), (8

In [9]:

#initialize graph
networkSubset, network, fwResult = initialize(link_file, trip_file, od_vols, origins, od_dic, links)

#find nash equilibrium
networkSubset, network, fwResult = calc_nash(networkSubset, network, fwResult)

#calculate latency

#export flow
#export latency